In [1]:
!pip install sentence-transformers
!pip install einop

In [4]:
from sentence_transformers import SentenceTransformer

# 1 โหลดโมเดล ทดสอบ 
# SentenceTransformers จะทำการ Download model ให้เราเอง อัตตโนมัติ
model = SentenceTransformer('BAAI/bge-m3')
#model = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True)
#model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')

# 2 สร้าง embeddings
sentences = ["สวัสดีครับ", "ประเทศไทย", "การเรียนรู้ python"]
embeddings = model.encode(sentences)
print(embeddings)

print(f"Embedding shape: {embeddings.shape}")  # Output: (768,) or (1024,) depending on the model
print(f"Embedding size: {embeddings.shape[0]} dimensions")

# 3 คำนวน embedding similarities

similarities = model.similarity(embeddings, embeddings)
print(similarities)

[[-0.04088951  0.02824118 -0.05546337 ... -0.00159979 -0.02763485
   0.00845535]
 [-0.02109072  0.04304685 -0.05400291 ...  0.00845333 -0.09080566
  -0.045714  ]
 [-0.04385765  0.02051526 -0.01604236 ... -0.01354666 -0.02235507
   0.04201256]]
Embedding shape: (3, 1024)
Embedding size: 3 dimensions
tensor([[1.0000, 0.4995, 0.4955],
        [0.4995, 1.0000, 0.3746],
        [0.4955, 0.3746, 1.0000]])


In [ ]:
!pip install psycopg2-binary
!pip install numpy

In [ ]:
import psycopg2 

conn = None  # Initialize to avoid NameError in finally
try:

    conn = psycopg2.connect(
        dbname="vectordb",
        user="postgres",
        password="postgres",
        host="localhost"
    )
    cursor = conn.cursor()
    print("Connection established.")
    # ... your database operations here ...
except Exception as err:
    print("Something went wrong.")
    print(err)
    
finally:
    # Cleanup resources in reverse order
    if 'cursor' in locals() and cursor:  # Check if cursor exists
        cursor.close()
    if conn:  # Check if connection exists
        conn.close()
    print("Resources released.")

## Create table documents

In [ ]:
import psycopg2
from psycopg2 import sql

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="vectordb",
    user="postgres",
    password="postgres",
    host="localhost"
)

try:
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        # Create table if it doesn't exist
        cur.execute("""
            CREATE TABLE IF NOT EXISTS documents (
                id serial PRIMARY KEY,
                content text,
                embedding vector(1024),
                created_at timestamptz DEFAULT now()
            );
        """)
        conn.commit()
        print("Table created successfully or already exists")
        
        # Verify table exists
        cur.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_name = 'documents'
            );
        """)
        exists = cur.fetchone()[0]
        print(f"Table exists: {exists}")
        
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    cur.close()  # ปิด Cursor ก่อน
    conn.close() # แล้วปิด Connection



## Chage to Function

In [ ]:
import psycopg2
from psycopg2 import sql

def create_vector_table(
    dbname="vectordb",
    user="postgres",
    password="postgres",
    host="localhost",
    table_name="items",
    vector_dim=3
):
    """
    Creates a PostgreSQL table with a vector column if it doesn't exist.
    
    Args:
        dbname (str): Database name
        user (str): Database user
        password (str): Database password
        host (str): Database host
        table_name (str): Name of the table to create
        vector_dim (int): Dimension for the vector column
        
    Returns:
        tuple: (success: bool, message: str)
    """
    conn = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host
        )
        
        with conn.cursor() as cur:
            # Ensure vector extension exists
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            
            # Create table with vector column
            cur.execute(sql.SQL("""
                CREATE TABLE IF NOT EXISTS {} (
                    id serial PRIMARY KEY,
                    content text,
                    embedding vector({}),
                    created_at timestamptz DEFAULT now(),
                    metadata jsonb
                );
            """).format(
                sql.Identifier(table_name),
                sql.Literal(vector_dim)
            ))
            
            conn.commit()
            
            # Verify table creation
            cur.execute(sql.SQL("""
                SELECT column_name, data_type 
                FROM information_schema.columns 
                WHERE table_name = {};
            """).format(sql.Literal(table_name)))
            
            columns = cur.fetchall()
            column_names = [col[0] for col in columns]
            
            message = f"Table '{table_name}' created with columns: {column_names}"
            return (True, message)
            
    except psycopg2.Error as e:
        error_msg = f"Database error occurred: {e}"
        if conn:
            conn.rollback()
        return (False, error_msg)
        
    except Exception as e:
        error_msg = f"Unexpected error: {e}"
        return (False, error_msg)
        
    finally:
        if conn:
            conn.close()

In [ ]:
success, message = create_vector_table(
        table_name="documents",
        vector_dim=1024  # For larger embeddings 768
)
print(f"Success: {success}")
print(f"Message: {message}")

In [ ]:
import psycopg2
from psycopg2 import sql

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-m3')
#model = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True)
#model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')

conn = psycopg2.connect(
    dbname="vectordb",
    user="postgres",
    password="postgres",
    host="localhost"
)

def add_documents(text: str) -> bool:
    """
    Adds a document to the PostgreSQL database with its text embedding.
    
    Args:
        text: The text content to be stored
        
    Returns:
        bool: True if successful, False if failed
    """
    conn = None
    try:
        # Generate embedding
        embedding = model.encode(text)
        
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname="vectordb",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        
        with conn.cursor() as cur:
            # Convert numpy array to PostgreSQL compatible format
            embedding_list = embedding.tolist()
            # Insert document and embedding
            cur.execute(
                "INSERT INTO documents (content, embedding) VALUES (%s, %s)",
                (text, embedding_list)
            )
            conn.commit()
        return True
        
    except Exception as e:
        print(f"Error adding document: {e}")
        if conn:
            conn.rollback()
        return False
        
    finally:
        if conn:
            conn.close()

# data สำหรับการ Embedding
documents = [
    "วันพุธ	1 มกราคม	วันขึ้นปีใหม่",
    "วันพุธ	12 กุมภาพันธ์	วันมาฆบูชา",
    "วันจันทร์	7 เมษายน	วันหยุดชดเชยวันจักรี",
    "วันจันทร์	14 เมษายน	วันสงกรานต์",
    "วันอังคาร	15 เมษายน	วันสงกรานต์",
    "วันพฤหัสบดี	1 พฤษภาคม	วันแรงงานแห่งชาติ ",
    "วันจันทร์	5 พฤษภาคม	วันหยุดชดเชยวันฉัตรมงคล",
    "วันศุกร์	9 พฤษภาคม	วันพืชมงคล (หยุดเฉพาะราชการ)",
    "วันจันทร์	12 พฤษภาคม	วันหยุดชดเชยวันวิสาขบูชา",
    "วันจันทร์	2 มิถุนายน	วันหยุดพิเศษ",
    "วันอังคาร	3 มิถุนายน	วันเฉลิมพระชนมพรรษาสมเด็จพระนางเจ้าสุทิดา",
    "วันพฤหัสบดี	10 กรกฎาคม	วันอาสาฬหบูชา",
    "วันศุกร์	11 กรกฎาคม	วันเข้าพรรษา (หยุดเฉพาะราชการ)",
    "วันจันทร์	28 กรกฎาคม	วันเฉลิมพระชนมพรรษาพระบาทสมเด็จพระเจ้าอยู่หัว",
    "วันจันทร์	11 สิงหาคม	วันหยุดพิเศษ",
    "วันอังคาร	12 สิงหาคม	วันแม่แห่งชาติ",
    "วันจันทร์	13 ตุลาคม	วันนวมินทรมหาราช",
    "วันพฤหัสบดี	23 ตุลาคม	วันปิยมหาราช",
    "วันศุกร์	5 ธันวาคม	วันคล้ายวันพระราชสมภพรัชกาลที่ 9 วันชาติ และ วันพ่อแห่งชาติ",
    "วันพุธ	10 ธันวาคม	วันรัฐธรรมนูญ",
    "วันพุธ	31 ธันวาคม	วันสิ้นปี",
]

for doc in documents:
    add_documents(doc)
    


In [ ]:
def print_five_documents():
    """
    Prints the first 5 documents from the database for verification
    """
    conn = None
    try:
        conn = psycopg2.connect(
            dbname="vectordb",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        
        
        with conn.cursor() as cur:
            cur.execute("SELECT id, content, embedding FROM documents LIMIT 5")
            rows = cur.fetchall()
            
            print("\nVerifying documents (first 5 rows):")
            print("-" * 50)
            for row in rows:
                print(f"ID: {row[0]}")
                print(f"Content: {row[1][:100]}...")  # Print first 100 chars of content
                print(f"Embedding length: {len(row[2])}")
                print("-" * 50)
                
    except Exception as e:
        print(f"Error fetching documents: {e}")
        
    finally:
        if conn:
            conn.close()
#Usage
print_five_documents()

In [9]:
import psycopg2
from psycopg2 import sql
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-m3')

def query_postgresql(query_text, k=5):
    query_embedding = model.encode(query_text).tolist()
    # print(query_embedding)

    # แปลง vector เป็น String
    query_embedding_str = "["+ ",".join(map(str,query_embedding)) + "]"
    # print(query_embedding_str)
    
    conn = None
    try:
        conn = psycopg2.connect(
            dbname="vectordb",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        # <=> คำนวนควาเหมือน ยิ่งน้อย ยิ่งเหมือน
        sql_query = """
            SELECT content, embedding <=> %s::vector AS similarity_score
            FROM  documents
            ORDER BY  similarity_score ASC
            LIMIT %s
        """
        
        with conn.cursor() as cur:
            cur.execute(sql_query,(query_embedding_str,k))
            results = cur.fetchall()
            
        return results
                
    except Exception as e:
        print(f"Error fetching documents: {e}")
        
    finally:
        if conn:
            conn.close()
#Usage
resutls = query_postgresql("วันหยุดเดือน มกราคม")
print(resutls)
                           

[('วันพุธ\t1 มกราคม\tวันขึ้นปีใหม่', 0.3361233077122753), ('วันจันทร์\t2 มิถุนายน\tวันหยุดพิเศษ', 0.43126433268360187), ('วันพุธ\t31 ธันวาคม\tวันสิ้นปี', 0.47097019418629704), ('วันจันทร์\t7 เมษายน\tวันหยุดชดเชยวันจักรี', 0.47764746778152767), ('วันจันทร์\t11 สิงหาคม\tวันหยุดพิเศษ', 0.47934623124163445)]


## Use Ollama

In [11]:
!pip install ollama

import ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [ollama]2m3/5 [pydantic]


## Test Ollama Function

In [ ]:
import ollama

prompt = "ประเทศไทยมีประชากรเท่าไหร่"  

response = ollama.chat(
    model="llama3",  
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response["message"]["content"])


In [18]:
import ollama

def generate_response(query_text):
    # 1. Retrieve relevant documents from PostgreSQL
    retrieved_docs = query_postgresql(query_text, 3)  # Fixed typo in variable name
    
    # 2. Prepare context from retrieved documents
    context = "\n".join([doc[0] for doc in retrieved_docs])
    print("=== Retrieved Context ===")
    print(context)
    
    # 3. Construct the prompt with context
    prompt = f"""Answer the question based on the following context:
{context}

Question: {query_text}
"""
    print("\n=== Generated Prompt ===")
    print(prompt)
    
    # 4. Generate response using Ollama
    try:
        response = ollama.chat(
            model="llama3",  # Corrected model name (no "llama3.2")
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions about Thailand."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # 5. Return the generated response
        generated_answer = response["message"]["content"]
        print("\n=== Generated Answer ===")
        print(generated_answer)
        return generated_answer
        
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

# Example usage
generate_response("วันสงกรานต์")

วันจันทร์	14 เมษายน	วันสงกรานต์
วันอังคาร	15 เมษายน	วันสงกรานต์
วันพุธ	1 มกราคม	วันขึ้นปีใหม่
Answer the question base on context: 
วันจันทร์	14 เมษายน	วันสงกรานต์
วันอังคาร	15 เมษายน	วันสงกรานต์
วันพุธ	1 มกราคม	วันขึ้นปีใหม่

Question วันสงกรานต์
